## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Cleaning Section

In [ ]:
import re

def clean_sentence(sentence):
    # Remove leading digits and any whitespace
    cleaned_sentence = re.sub(r'^\d+\s*', '', sentence)
    # Remove any leading or trailing whitespace
    cleaned_sentence = cleaned_sentence.strip()
    return cleaned_sentence


## Uploading Correct Sentences(sentences) and wrong Sentences(labels).

In [ ]:
# Read the file and store sentences in a list
y = []
with open("/content/drive/My Drive/ara-eg_newscrawl-OSIAN_2018_10K-sentences.txt", "r", encoding="utf-8") as file:
    for line in file:
        cleaned_line = clean_sentence(line)
        if cleaned_line:  # Check if the line is not empty after cleaning
            y.append(cleaned_line)

In [ ]:
# Read the file and store sentences in a list
x = []
with open("/content/drive/My Drive/ara-eg_newscrawl-OSIAN_2018_10K-labels.txt", "r", encoding="utf-8") as file:
    for line in file:
        cleaned_line = clean_sentence(line)
        if cleaned_line:  # Check if the line is not empty after cleaning
            x.append(cleaned_line)

In [ ]:
import re

def clean_sentence(sentence):
    # Remove English words using regular expression
    cleaned_sentence = re.sub(r'\b[a-zA-Z]+\b', '', sentence)
    # Remove numbers and signs using regular expression
    cleaned_sentence = re.sub(r'\d+|\W+', ' ', cleaned_sentence)
    # Remove punctuation marks
    cleaned_sentence = re.sub(r'[^\w\s]', '', cleaned_sentence)
    # Normalize diacritics (fatha, kasra, damma)
    cleaned_sentence = cleaned_sentence.replace('ً', '').replace('ٌ', '').replace('ٍ', '').replace('َ', '').replace('ُ', '').replace('ِ', '')
    # Remove special characters
    cleaned_sentence= re.sub(r'[^\u0600-\u06FF\s\d]', '', cleaned_sentence)
    # Remove numbers
    cleaned_sentence= re.sub(r'\d+', '', cleaned_sentence)
    # Normalize whitespace
    cleaned_sentence= re.sub(r'\s+', ' ', cleaned_sentence.strip())
    # Remove extra whitespace
    cleaned_sentence = ' '.join(cleaned_sentence.split())
    return cleaned_sentence

# Clean each sentence in the list
x = [clean_sentence(sentence) for sentence in x]
# Clean each sentence in the list
y = [clean_sentence(sentence) for sentence in y]

## Get the 1st 5 sentence from both correct and wrong sentences.

In [ ]:
print(x[:5])
print(y[:5])

['مليارات و مليون درهم خلال تكل الفترة', 'معلومات خاطئة نعتقد للأسنان جيدة للأسنان معلومات خاطئة نعتقد أنها جيدة للأسنان', 'مسلسل يونس ولد بطولة عمرو سعد وسيعرض على قناة مصر دراما المستقبل السومرية', 'الدكتورة منى بنت عبدالله بن سعيد آل مشيط', 'صودا الخبز هو معجون طبيعي للأسنان حثي ينصح بخلط ربع ملعقة صغيرة من صودا الخبز مع الماء وغسل الأسنان ببه']
['مليارات و مليون درهم خلال تلك الفترة', 'معلومات خاطئة نعتقد أنها جيدة للأسنان معلومات خاطئة نعتقد أنها جيدة للأسنان', 'مسلسل يونس ولد فضة بطولة عمرو سعد وسيعرض على قناة مصر دراما المستقبل السومرية', 'الدكتورة منى بنت عبدالله بن سعيد آل مشيط', 'صودا الخبز هو معجون طبيعي للأسنان حيث ينصح بخلط ربع ملعقة صغيرة من صودا الخبز مع الماء وغسل الأسنان ببه']


## Import important libraries.

In [ ]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForMaskedLM, AdamW
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

## Getting seek into the model, Injoy..

In [ ]:
# Load the pre-trained AraBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/717k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Add special tokens to the sentences
max_length = 32  # Define maximum sequence length

In [ ]:
# Tokenize the sentences (assuming 'x' and 'y' are your input and target sequences)
tokenized_x = tokenizer(x, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
tokenized_y = tokenizer(y, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

In [ ]:
# Convert tokenized sentences to PyTorch tensors
input_ids = tokenized_x.input_ids
labels = tokenized_y.input_ids

# Create a PyTorch Dataset
dataset = TensorDataset(input_ids, labels)

In [ ]:
# Define training parameters
learning_rate = 5e-5
num_epochs = 5
batch_size = 8
num_folds = 5  # Number of folds for cross-validation

In [ ]:
# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
# Initialize lists to store metrics for each fold
train_losses = []
val_losses = []
test_losses = []
train_accuracies = []
val_accuracies = []
test_accuracies = []

In [ ]:
# Cross-validation loop
for fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold+1}/{num_folds}")

    # Split dataset into train and test sets for this fold
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)

    # Create DataLoaders for train and test sets
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define optimizer and loss function
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Train the model for this fold
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        for batch in train_dataloader:
            # Move batch to device
            batch = tuple(t.to(device) for t in batch)
            input_ids, labels = batch

            # Forward pass
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Calculate predictions and true labels
            preds = logits.argmax(dim=-1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        # Convert the train_labels and train_preds lists to NumPy arrays
        train_labels = np.array(train_labels)
        train_preds = np.array(train_preds)

        # Calculate average training loss for the epoch
        train_avg_loss = train_loss / len(train_dataloader)

        # Calculate the accuracy score
        train_accuracy = accuracy_score(train_labels.ravel(), train_preds.ravel())
        print(f"Fold {fold+1}, Epoch {epoch+1}, Train Loss: {train_avg_loss}, Train Accuracy: {train_accuracy}")

    # Test for this fold
    model.eval()
    test_loss = 0
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in test_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, labels = batch

            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            test_loss += loss.item()

            # Calculate predictions and true labels
            preds = logits.argmax(dim=-1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    # Convert the test_labels and test_preds lists to NumPy arrays
    test_labels = np.array(test_labels)
    test_preds = np.array(test_preds)

    # Calculate average test loss for the fold
    test_avg_loss = test_loss / len(test_dataloader)

    # Calculate the accuracy score
    test_accuracy = accuracy_score(test_labels.ravel(), test_preds.ravel())
    print(f"Fold {fold+1}, Test Loss: {test_avg_loss}, Test Accuracy: {test_accuracy}")

    # Append metrics for this fold to the lists
    test_losses.append(test_avg_loss)
    test_accuracies.append(test_accuracy)

# Print average test loss and accuracy across all folds
print(f"Average Test Loss: {np.mean(test_losses)}, Average Test Accuracy: {np.mean(test_accuracies)}")


Fold 1/5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Fold 1, Epoch 1, Train Loss: 1.5191736900806427, Train Accuracy: 0.73163671875
Fold 1, Epoch 2, Train Loss: 1.0432270367890597, Train Accuracy: 0.7752890625
Fold 1, Epoch 3, Train Loss: 0.7492960204929113, Train Accuracy: 0.82969921875
Fold 1, Epoch 4, Train Loss: 0.5369095611236989, Train Accuracy: 0.87153125
Fold 1, Epoch 5, Train Loss: 0.40296816068701447, Train Accuracy: 0.89820703125
Fold 1, Test Loss: 1.4670731830000878, Test Accuracy: 0.7973125
Fold 2/5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Fold 2, Epoch 1, Train Loss: 0.577788025084883, Train Accuracy: 0.87509765625
Fold 2, Epoch 2, Train Loss: 0.34849737108498813, Train Accuracy: 0.9137578125
Fold 2, Epoch 3, Train Loss: 0.26100907327234746, Train Accuracy: 0.93108203125
Fold 2, Epoch 4, Train Loss: 0.20169293646980077, Train Accuracy: 0.9448203125
Fold 2, Epoch 5, Train Loss: 0.1624084329821635, Train Accuracy: 0.95409375
Fold 2, Test Loss: 0.4265996302496642, Test Accuracy: 0.9123125
Fold 3/5


...

Fold 3, Epoch 1, Train Loss: 0.2294270624020137, Train Accuracy: 0.9419765625
Fold 3, Epoch 2, Train Loss: 0.15946081203175708, Train Accuracy: 0.9567265625
Fold 3, Epoch 3, Train Loss: 0.12032410067424644, Train Accuracy: 0.9662578125
Fold 3, Epoch 4, Train Loss: 0.09967291083699092, Train Accuracy: 0.9723984375
Fold 3, Epoch 5, Train Loss: 0.08295027286699042, Train Accuracy: 0.97697265625
Fold 3, Test Loss: 0.1523933961223811, Test Accuracy: 0.96015625
Fold 4/5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Fold 4, Epoch 1, Train Loss: 0.1242829478806816, Train Accuracy: 0.96719921875
Fold 4, Epoch 2, Train Loss: 0.0850342912771739, Train Accuracy: 0.9768046875
Fold 4, Epoch 3, Train Loss: 0.06704074032942299, Train Accuracy: 0.9820546875
Fold 4, Epoch 4, Train Loss: 0.0697156179837184, Train Accuracy: 0.9814921875
Fold 4, Epoch 5, Train Loss: 0.06043793439131696, Train Accuracy: 0.98402734375
Fold 4, Test Loss: 0.07152686445764266, Test Accuracy: 0.979953125
Fold 5/5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Fold 5, Epoch 1, Train Loss: 0.07268637857493013, Train Accuracy: 0.98083984375
Fold 5, Epoch 2, Train Loss: 0.04792855208704714, Train Accuracy: 0.987265625
Fold 5, Epoch 3, Train Loss: 0.05662169781263219, Train Accuracy: 0.9853671875
Fold 5, Epoch 4, Train Loss: 0.06116278507676907, Train Accuracy: 0.98465625
Fold 5, Epoch 5, Train Loss: 0.05472746467031538, Train Accuracy: 0.98653515625
Fold 5, Test Loss: 0.03394576233671978, Test Accuracy: 0.992125
Average Test Loss: 0.4303077672332991, Average Test Accuracy: 0.9283718749999998


## Try the model after cross validation by testing it.

In [ ]:
import torch
from transformers import AutoTokenizer
from torch.nn.functional import softmax

def auto_correct_sentence(model, tokenizer, sentence, device):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

    # Move inputs to device
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Forward pass through the model
    outputs = model(**inputs)
    logits = outputs.logits

    # Get probabilities using softmax
    probabilities = softmax(logits, dim=2)

    # Get the predicted labels
    _, predicted_labels = torch.max(probabilities, dim=2)

    # Decode predicted labels to tokens
    corrected_tokens = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in predicted_labels]

    # Join tokens to form corrected sentence
    corrected_sentence = " ".join(corrected_tokens)

    return corrected_sentence

# Example usage
def auto_correct_sentences(model, tokenizer, sentences, device):
    corrected_sentences = []
    for sentence in sentences:
        corrected_sentence = auto_correct_sentence(model, tokenizer, sentence, device)
        corrected_sentences.append(corrected_sentence)
    return corrected_sentences

# Example usage
sentences = ["صودا الخبز هو معجون طبيعي للأسنان حثي ينصح بخلط ربع ملعقة صغيرة من صودا الخبز مع الماء وغسل الأسنان ببه"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set device
corrected_sentences = auto_correct_sentences(model, tokenizer, sentences, device)
print("Original sentence:", sentences[0])
print("Corrected sentence:", corrected_sentences[0])


Original sentence: صودا الخبز هو معجون طبيعي للأسنان حثي ينصح بخلط ربع ملعقة صغيرة من صودا الخبز مع الماء وغسل الأسنان ببه
Corrected sentence: صودا الخبز هو معجون طبيعي للأسنان حيث ينصح بخلط ربع ملعقة صغيرة من صودا الخبز مع الماء وحمر الأسنان ببه ب
